# PB016: Artificial intelligence I, labs 9 - Examples of working with knowledge representation frameworks in code

This week's topic is representation of knowledge using ontologies and graphical models. We'll focus namely on:

1. __Working with a sample ontology__
2. __Working with a sample Bayesian network__

---

## 1. Working with a sample [ontology](https://en.wikipedia.org/wiki/Ontology_%28information_science%29) - pizza time!

![pizza](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/pizza.jpg)

__Basic facts__
- Ontologies are machine-readable formalizations of general or domain knowledge. They typically address the following:
  - representation, formal naming and definition of general categories (also called concepts or classes) and sets of specific entities that fall under these categories (these entities are usually called individuals, instances or objects);
  - properties of the categories and individual entities;
  - relationships between categories and individuals (super- and sub-classes, instances of classes);
  - metadata and annotations that do not affect the logical interpretation (i.e., formal meaning) of the elements of the ontology, but can be useful for people and applications working with the ontology.
- The formal basis of ontologies is usually propositional and/or first-order predicate logic (FOL), or rather its decidable subsets and/or extensions (e.g., fuzzy or probabilistic).
- No tractable formalism uses the full range of possibilities of FOL (even the less expressive variants of languages ​​for the representation of ontologies are relatively demanding in terms of reasoning complexity).
- In these labs we will deal with ontologies represented in the [OWL](https://en.wikipedia.org/wiki/Web_Ontology_Language) language, which is the "flagship" of the [W3C standard](https://www.w3.org/standards/semanticweb/) for [semantic web](https://en.wikipedia.org/wiki/Semantic_Web), based on [descriptive logic](https://en.wikipedia.org/wiki/Description_logic) (an extensively researched and relatively widely used formalism, especially in the [knowledge representation and reasoning](https://en.wikipedia.org/wiki/Knowledge_representation_and_reasoning) community).

### __Installing [owlready2](https://owlready2.readthedocs.io/en/latest/)__

In [1]:
# installation and import of a package for working with ontologies;
# for details, see: https://owlready2.readthedocs.io/

!pip install owlready2

Defaulting to user installation because normal site-packages is not writeable
  Using cached owlready2-0.47.tar.gz (27.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.47-cp311-cp311-linux_x86_64.whl size=24582288 sha256=7a972da0276db8fa6acbe76399b6b359550dcd206ea273369d208109d17d6fc5
  Stored in directory: /home/jindmen/.cache/pip/wheels/25/9a/a3/fb1ac6339caa859c8bb18d685736168b0b51d851af13d81d52
Successfully built owlready2

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


### __Loading a sample [pizza ontology](https://protege.stanford.edu/ontologies/pizza/pizza.owl)__

In [2]:
from owlready2 import *

PIZZA_URL = 'https://protege.stanford.edu/ontologies/pizza/pizza.owl'
PIZZA_NAMESPACE = 'http://www.co-ode.org/ontologies/pizza/pizza.owl'
onto_ns = get_namespace(PIZZA_NAMESPACE)

onto = get_ontology(PIZZA_URL).load()

### __Listing items in the ontology__
- Ontologies behave like classic Python objects in `owlready2`.
- Their elements can be accessed using the "dot" notation, it is possible to iterate over them, their methods can be called, some objects have operators defined, etc.
- Searching for explicitly defined elements is possible using the `search ()` function, which does not use any inference, but is still relatively [powerful](https://owlready2.readthedocs.io/en/latest/onto.html#simple-queries) (it supports searching for sub-classes, wild cards, filtering by element properties, nested search, etc.).

In [3]:
print('List of classes and their direct sub-classes:\n')

# iterating over the top-level ontology classes
for onto_class in onto.classes():
    print(onto_class, '(%s)' % onto_class.iri)

    # iterating over the sub-classes of the current class, their listing
    for sub_class in onto.search(subclass_of=onto_class):
        print('  sub-class:', sub_class)

print('\n'+'-'*80+'\n')
print('List of individuals in the loaded ontology:\n')

# iterating over the ontology individuals
for individual in onto.individuals():
    print(individual)

print('\n'+'-'*80+'\n')
print('List of object properties in the loaded ontology:\n')

# iterating over the object properties in the ontology
for onto_property in onto.object_properties():
    print(onto_property)

List of classes and their direct sub-classes:

pizza.Pizza (http://www.co-ode.org/ontologies/pizza/pizza.owl#Pizza)
  sub-class: pizza.Pizza
  sub-class: pizza.NamedPizza
  sub-class: pizza.UnclosedPizza
  sub-class: pizza.American
  sub-class: pizza.AmericanHot
  sub-class: pizza.Cajun
  sub-class: pizza.Capricciosa
  sub-class: pizza.Caprina
  sub-class: pizza.Fiorentina
  sub-class: pizza.FourSeasons
  sub-class: pizza.FruttiDiMare
  sub-class: pizza.Giardiniera
  sub-class: pizza.LaReine
  sub-class: pizza.Margherita
  sub-class: pizza.Mushroom
  sub-class: pizza.Napoletana
  sub-class: pizza.Parmense
  sub-class: pizza.PolloAdAstra
  sub-class: pizza.PrinceCarlo
  sub-class: pizza.QuattroFormaggi
  sub-class: pizza.Rosa
  sub-class: pizza.Siciliana
  sub-class: pizza.SloppyGiuseppe
  sub-class: pizza.Soho
  sub-class: pizza.Veneziana
pizza.PizzaBase (http://www.co-ode.org/ontologies/pizza/pizza.owl#PizzaBase)
  sub-class: pizza.PizzaBase
  sub-class: pizza.DeepPanBase
  sub-class:

### __Creating a new pizza (olomuciana)__
- The pizza will be represented by a new `Olomuciana` class.
- The base will be thin (Neapolitan style, i.e., `ThinAndCrispyBase`) and the pizza will have the following toppings:
  - `TomatoTopping` (existing);
  - `OnionTopping` (existing);
  - `PaprikaTopping` (non-existent, must be created);
  - `TvargleTopping` (non-existent, must be created).
- To be upfront towards the customers, it would probably also be a good idea to create a new `PungencyLevel` category (similar to the existing` Spiciness` class), which can have one of the following values:
  - `BlandAndBoring`;
  - `MildlyAromatic`;
  - `PrettyRipe`;
  - `IsThereARottingCorpseInTheCellarOrWhat`.
- Finally, one also needs to create a new `hasPungency` property that assigns `PungencyLevel` to `Pizza` objects.

In [4]:
# we work with the loaded ontology
with onto:

    # adding new classes defined as Python classes/objects, inheriting from
    # their corresponding super-classes - access to existing classes is
    # mediated via the object "onto_ns", which is a specific namespace (de
    # facto a domain, within which the given names of classes, individuals,
    # properties, etc. apply), in which they are ontology elements defined

    class PaprikaTopping(onto_ns.VegetableTopping):
        # inherits from the existing class of vegetable toppings
        pass

    class TvargleTopping(onto_ns.CheeseTopping):
        # inherits from the existing class of cheese toppings
        pass

    class PungencyLevel(Thing):
        # inherits from a special "top" class - everything is a type of "Thing"
        pass

    class BlandAndBoring(PungencyLevel):
        # inherits from the new class
        pass

    class MildlyAromatic(PungencyLevel):
        # inherits from the new class
        pass

    class PrettyRipe(PungencyLevel):
        # inherits from the new class
        pass

    class IsThereARottingCorpseInTheCellarOrWhat(PungencyLevel):
        # inherits from the new class
        pass

    # defining the mutual disjointness of the various "PungencyLevel" values

    AllDisjoint([BlandAndBoring, MildlyAromatic, PrettyRipe,
                 IsThereARottingCorpseInTheCellarOrWhat])

    # adding the new property that assigns the pungency to particular pizzas
    # NOTE: the operator >> defines the domain and range of the property,
    # respectively

    class hasPungency(onto_ns.Pizza >> PungencyLevel):
        pass

    # finally, we create the class representing the new pizza

    class Olomuciana(onto_ns.Pizza):
        # the base type
        hasBase = [onto_ns.ThinAndCrispyBase]
        # specific ingedients assigned to the pizza using a previously
        # existing property
        hasTopping = [onto_ns.TomatoTopping, TvargleTopping,
                      onto_ns.OnionTopping, PaprikaTopping]
        # specific pungency level
        hasPungency = [IsThereARottingCorpseInTheCellarOrWhat]


# checking the newly created elements
print('Our new pizza (general class)       :\n ', Olomuciana,
      '(%s)' % Olomuciana.iri)
# creating an instance of the new pizza
straight_from_the_oven = Olomuciana('freshly_baked_Olomuciana')
print('Our new pizza (specific baked pizza):\n ', straight_from_the_oven,
      '(%s)' % straight_from_the_oven.iri)

# adding the newly created elements to the namespace of the original Pizza
# ontology
for element in [PaprikaTopping, TvargleTopping, PungencyLevel, BlandAndBoring,
                MildlyAromatic, PrettyRipe,
                IsThereARottingCorpseInTheCellarOrWhat, hasPungency,
                Olomuciana, straight_from_the_oven]:
    element.namespace = onto_ns

# listing the sub-classes of the original and newly created classes of the
# pizza features
print('Sub-classes of the pizza feature classes:')
print('- sub-classes of the original "Spiciness" class:')
for subclass in onto_ns.Spiciness.subclasses():
    print('   ', subclass, '(%s)' % subclass.iri)
print('- sub-classes of the new "PungencyLevel" class:')
for subclass in PungencyLevel.subclasses():
    print('   ', subclass, '(%s)' % subclass.iri)

Our new pizza (general class)       :
  pizza.Olomuciana (http://www.co-ode.org/ontologies/pizza/Olomuciana)
Our new pizza (specific baked pizza):
  pizza.freshly_baked_Olomuciana (http://www.co-ode.org/ontologies/pizza/freshly_baked_Olomuciana)
Sub-classes of the pizza feature classes:
- sub-classes of the original "Spiciness" class:
    pizza.Mild (http://www.co-ode.org/ontologies/pizza/pizza.owl#Mild)
    pizza.Hot (http://www.co-ode.org/ontologies/pizza/pizza.owl#Hot)
    pizza.Medium (http://www.co-ode.org/ontologies/pizza/pizza.owl#Medium)
- sub-classes of the new "PungencyLevel" class:
    pizza.BlandAndBoring (http://www.co-ode.org/ontologies/pizza/pizza.owl#BlandAndBoring)
    pizza.MildlyAromatic (http://www.co-ode.org/ontologies/pizza/pizza.owl#MildlyAromatic)
    pizza.PrettyRipe (http://www.co-ode.org/ontologies/pizza/pizza.owl#PrettyRipe)
    pizza.IsThereARottingCorpseInTheCellarOrWhat (http://www.co-ode.org/ontologies/pizza/pizza.owl#IsThereARottingCorpseInTheCellarOrWh

### __Defining a class restriction__
- Restrictions specify the meaning of classes beyond their position in a taxonomy, etc. - it is possible, for instance, to define restrictions on the cardinality of class properties, etc.
  - An example is a simple pizza (`SimplePizza`), which has a maximum of three toppings.

In [5]:
with onto:
    class SimplePizza(onto_ns.Pizza): # inherits from an existing class Pizza
        # it's equivalent to an intersection of all pizza classes a classes that have up to three toppings
        equivalent_to = [onto_ns.Pizza & onto_ns.hasTopping.max(3,onto_ns.PizzaTopping)]

### __Defining a property restriction__
- Creating a new `hasDiameter` property specifying the diameter of a specific pizza.
- Ensuring that the pizza can only have one diameter (e.g., one of 18cm, 23cm, 28cm or 33cm).

A note on the solution: The `OneOf` construct is used to explicitly define instances of the` Diameter` class, which is the range of the `hasDiameter` property. Alternatively, one could use the built-in OWL type `FunctionalProperty`, which defines properties with just one possible value. For more details, see [this](https://owlready2.readthedocs.io/en/latest/restriction.html#one-of-constructs) or [this](https://owlready2.readthedocs.io/en/latest/properties.html#functional-and-inverse-functional-properties) part of the Owlready2 manual.

In [5]:
with onto:

    # creating a new pizza diameter class

    class Diameter(Thing):
        pass

    # defining the individuals representing different diameters

    small = Diameter('small=18cm')
    medium = Diameter('medium=23cm')
    large = Diameter('large=28cm')
    monster = Diameter('monster=33cm')

    # explicit restriction of the diameter class to a list of possible individuals

    Diameter.is_a.append(OneOf([small, medium, large, monster]))

    # defining the functional property hasDiameter

    class hasDiameter(ObjectProperty, FunctionalProperty):
        domain = [onto_ns.Pizza]
        range = [Diameter]

# listing the details of the new pizza
print('New class:', onto.Diameter)
small_pizza = onto_ns.Pizza('small_pizza')
print('New pizza instance:', small_pizza)
small_pizza.hasDiameter = small
print('Diameter of the new pizza instance:', small_pizza.hasDiameter)

New class: pizza.Diameter
New pizza instance: pizza.small_pizza
Diameter of the new pizza instance: pizza.small=18cm


### __Classification of the pizza ontology by the [HermiT](http://www.hermit-reasoner.com/) reasoner__
- Derivation of implicit facts based on description logic and materialization of the corresponding updated taxonomic structure.
- Possible invalidation of inconsistent classes by adding an explicit statement that they are equivalent to "nothing" (but only if they are empty, i.e., if they have no instances; otherwise the whole process will fail).

In [6]:
with onto:
    sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/jindmen/.local/lib/python3.11/site-packages/owlready2/hermit:/home/jindmen/.local/lib/python3.11/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpuu9ka660
* Owlready2 * HermiT took 2.635690450668335 seconds
* Owlready * Equivalenting: pizza.VegetarianPizzaEquivalent1 pizza.VegetarianPizzaEquivalent2
* Owlready * Equivalenting: pizza.VegetarianPizzaEquivalent2 pizza.VegetarianPizzaEquivalent1
* Owlready * Equivalenting: pizza.CheeseyVegetableTopping owl.Nothing
* Owlready * Equivalenting: pizza.IceCream owl.Nothing
* Owlready * Equivalenting: pizza.SpicyPizza pizza.SpicyPizzaEquivalent
* Owlready * Equivalenting: pizza.SpicyPizzaEquivalent pizza.SpicyPizza
* Owlready * Reparenting pizza.ThinAndCrispyPizza: {owl.Thing} => {pizza.Pizza}
* Owlready * Reparenting pizza.NonVegetarianPizza: {owl.Thing} => {pizza.Pizza}
* Owlready * Reparenting pizza.InterestingPizza: {owl.T

## 2. Working with a sample Bayesian network

A [Bayesian network](https://en.wikipedia.org/wiki/Bayesian_network), Bayes network, belief network, Bayes(ian) model or probabilistic directed acyclic graphical model is defined as follows:
- It is a probabilistic graphical model (a type of statistical model) that represents a set of random variables and their conditional dependencies via a directed acyclic graph (DAG).
- The model is mostly used to represent causal relationship between the random variables.
- Bayesian Networks are parameterized using [Conditional Probability Distributions](https://en.wikipedia.org/wiki/Conditional_probability_distribution) (CPD).
- Each node in the network is parameterized using $P(node|Pa(node))$ where $Pa(node)$ represents the parents of node in the network.

### __A sample Bayesian network__
- student attribute (intelligence - I)
- course attribute (difficulty - D)
- grade (G)
- recommendation letter attribute (positive/negative - L)
- student's overall SAT score (positive/negative - S)

![Bayes Netwok](https://www.fi.muni.cz/~novacek/courses/pb016/labs/img/BN-example.png)

### __Working with the sample network - installing [pgmpy](https://github.com/pgmpy/pgmpy)__

In [1]:
!pip install pgmpy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 24.0 MB/s eta 0:00:00m eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 14.5 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 19.4 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.4 MB/s eta 0:00:000:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.6 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 20.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 24.9 MB/s eta 0:00:00m eta 0:00:01

### __Creating the model__

In [2]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD

# Defining the model structure. We can define the network by just passing a
# list of edges.
model = BayesianNetwork([('D', 'G'), ('I', 'G'), ('G', 'L'), ('I', 'S')])

# Defining individual CPDs.
cpd_d = TabularCPD(variable='D', variable_card=2, values=[[0.6], [0.4]])
cpd_i = TabularCPD(variable='I', variable_card=2, values=[[0.7], [0.3]])

# The representation of CPD in pgmpy is a bit different than the CPD shown in
# the above picture. In pgmpy the colums are the evidences and rows are the
# states of the variable. So the grade CPD is represented like this:
#
#    +---------+---------+---------+---------+---------+
#    | diff    | intel_0 | intel_0 | intel_1 | intel_1 |
#    +---------+---------+---------+---------+---------+
#    | intel   | diff_0  | diff_1  | diff_0  | diff_1  |
#    +---------+---------+---------+---------+---------+
#    | grade_0 | 0.3     | 0.05    | 0.9     | 0.5     |
#    +---------+---------+---------+---------+---------+
#    | grade_1 | 0.4     | 0.25    | 0.08    | 0.3     |
#    +---------+---------+---------+---------+---------+
#    | grade_2 | 0.3     | 0.7     | 0.02    | 0.2     |
#    +---------+---------+---------+---------+---------+

cpd_g = TabularCPD(variable='G', variable_card=3,
                   values=[[0.3, 0.05, 0.9,  0.5],
                           [0.4, 0.25, 0.08, 0.3],
                           [0.3, 0.7,  0.02, 0.2]],
                  evidence=['I', 'D'],
                  evidence_card=[2, 2])

cpd_l = TabularCPD(variable='L', variable_card=2,
                   values=[[0.1, 0.4, 0.99],
                           [0.9, 0.6, 0.01]],
                   evidence=['G'],
                   evidence_card=[3])

cpd_s = TabularCPD(variable='S', variable_card=2,
                   values=[[0.95, 0.2],
                           [0.05, 0.8]],
                   evidence=['I'],
                   evidence_card=[2])

# Associating the CPDs with the network
model.add_cpds(cpd_d, cpd_i, cpd_g, cpd_l, cpd_s)

# check_model checks for the network structure and CPDs and verifies that the
# CPDs are correctly defined and sum to 1.
model.check_model()

True

### __Creating the model using state names__

In [3]:
# CPDs can also be defined using the state names of the variables. If the state
# names are not provided like in the previous example, pgmpy will automatically
# assign names as: 0, 1, 2, ....

cpd_d_sn = TabularCPD(variable='D', variable_card=2, values=[[0.6], [0.4]],
                      state_names={'D': ['Easy', 'Hard']})
cpd_i_sn = TabularCPD(variable='I', variable_card=2, values=[[0.7], [0.3]],
                      state_names={'I': ['Dumb', 'Intelligent']})
cpd_g_sn = TabularCPD(variable='G', variable_card=3,
                      values=[[0.3, 0.05, 0.9,  0.5],
                              [0.4, 0.25, 0.08, 0.3],
                              [0.3, 0.7,  0.02, 0.2]],
                      evidence=['I', 'D'],
                      evidence_card=[2, 2],
                      state_names={'G': ['A', 'B', 'C'],
                                   'I': ['Dumb', 'Intelligent'],
                                   'D': ['Easy', 'Hard']})

cpd_l_sn = TabularCPD(variable='L', variable_card=2,
                      values=[[0.1, 0.4, 0.99],
                              [0.9, 0.6, 0.01]],
                      evidence=['G'],
                      evidence_card=[3],
                      state_names={'L': ['Bad', 'Good'],
                                   'G': ['A', 'B', 'C']})

cpd_s_sn = TabularCPD(variable='S', variable_card=2,
                      values=[[0.95, 0.2],
                              [0.05, 0.8]],
                      evidence=['I'],
                      evidence_card=[2],
                      state_names={'S': ['Bad', 'Good'],
                                   'I': ['Dumb', 'Intelligent']})

# These defined CPDs can be added to the model. Since, the model already has
# CPDs associated to variables, it will show warning that pmgpy is now
# replacing those CPDs with the new ones.
model.add_cpds(cpd_d_sn, cpd_i_sn, cpd_g_sn, cpd_l_sn, cpd_s_sn)
model.check_model()

True

### __Exploring the created model__

In [4]:
# We can now call some methods on the BayesianModel object.
model.get_cpds()

[<TabularCPD representing P(D:2) at 0x7675865b3050>,
 <TabularCPD representing P(I:2) at 0x7675871c85d0>,
 <TabularCPD representing P(G:3 | I:2, D:2) at 0x7675865b3610>,
 <TabularCPD representing P(L:2 | G:3) at 0x7675865b3750>,
 <TabularCPD representing P(S:2 | I:2) at 0x7675865b3dd0>]

In [5]:
# Printing a CPD which doesn't have state names defined.
print(cpd_g)

+------+------+------+------+------+
| I    | I(0) | I(0) | I(1) | I(1) |
+------+------+------+------+------+
| D    | D(0) | D(1) | D(0) | D(1) |
+------+------+------+------+------+
| G(0) | 0.3  | 0.05 | 0.9  | 0.5  |
+------+------+------+------+------+
| G(1) | 0.4  | 0.25 | 0.08 | 0.3  |
+------+------+------+------+------+
| G(2) | 0.3  | 0.7  | 0.02 | 0.2  |
+------+------+------+------+------+


In [6]:
# Printing a CPD with it's state names defined.
print(model.get_cpds('G'))

+------+---------+---------+----------------+----------------+
| I    | I(Dumb) | I(Dumb) | I(Intelligent) | I(Intelligent) |
+------+---------+---------+----------------+----------------+
| D    | D(Easy) | D(Hard) | D(Easy)        | D(Hard)        |
+------+---------+---------+----------------+----------------+
| G(A) | 0.3     | 0.05    | 0.9            | 0.5            |
+------+---------+---------+----------------+----------------+
| G(B) | 0.4     | 0.25    | 0.08           | 0.3            |
+------+---------+---------+----------------+----------------+
| G(C) | 0.3     | 0.7     | 0.02           | 0.2            |
+------+---------+---------+----------------+----------------+


In [7]:
model.get_cardinality('G')

3

### __Inference using the model__

In [15]:
from pgmpy.inference import VariableElimination
infer = VariableElimination(model)

# Grade probability when an intelligent student enrolls into an easy course
print(infer.query(['G'], evidence={'D': 'Easy', 'I': 'Intelligent'}))
print(infer.query(['L'], evidence={'D': 'Easy', 'I': 'Intelligent'}))
print(infer.query(['L'], evidence={'D': 'Hard', 'I': 'Intelligent'}))

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(A) |   0.9000 |
+------+----------+
| G(B) |   0.0800 |
+------+----------+
| G(C) |   0.0200 |
+------+----------+
+---------+----------+
| L       |   phi(L) |
+=========+==========+
| L(Bad)  |   0.1418 |
+---------+----------+
| L(Good) |   0.8582 |
+---------+----------+
+---------+----------+
| L       |   phi(L) |
+=========+==========+
| L(Bad)  |   0.3680 |
+---------+----------+
| L(Good) |   0.6320 |
+---------+----------+


---

#### _Final note_ - the materials used in this notebook are adapted from original works authored as follows:
- Example of pizza ontology:
 - Retrieved from [Protege](https://protege.stanford.edu/)
 - Author: Nick Drummond, Alan Rector, Matthew Horridge, Chris Wroe, Robert Stevens
 - License: [CC BY 3.0](https://creativecommons.org/licenses/by/3.0/)
- Pizza picture:
 - Retrieved from [Wikimedia Commons](https://commons.wikimedia.org/wiki/File:Eq_it-na_pizza-margherita_sep2005_sml.jpg)
 - Author: [Valerio Capello](https://en.wikipedia.org/wiki/User:ElfQrin)
 - License: [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/deed.en), [GNU Free Documenation License](https://en.wikipedia.org/wiki/en:GNU_Free_Documentation_License)
- The `pgmpy`-related contents:
 - Adapted from the `pgmpy` [Github site](https://github.com/pgmpy/pgmpy)
 - Author: The `pgmpy` [contributors](https://github.com/pgmpy/pgmpy/graphs/contributors)
 - License: [MIT](https://mit-license.org/)